In [1]:
pip install gradio

     ---------------------------------------- 0.0/43.0 kB ? eta -:--:--
     ---------------------------------------- 43.0/43.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/54.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/54.1 MB 6.4 MB/s eta 0:00:09
   ---------------------------------------- 0.3/54.1 MB 4.1 MB/s eta 0:00:14
   ---------------------------------------- 0.4/54.1 MB 3.6 MB/s eta 0:00:15
   ---------------------------------------- 0.6/54.1 MB 3.6 MB/s eta 0:00:15
    --------------------------------------- 0.7/54.1 MB 3.6 MB/s eta 0:00:15
    --------------------------------------- 0.8/54.1 MB 3.2 MB/s eta 0:00:17
    --------------------------------------- 1.0/54.1 MB 3.2 MB/s eta 0:00:17
    --------------------------------------- 1.1/54.1 MB 3.1 MB/s eta 0:00:17
    --------------------------------------- 1.2/54.1 MB 3.2 MB/s eta 0:00:17
    --------------------------------------- 1.3/54.1 MB 3.0 MB/s eta 0:00:18
   - ----------

In [17]:
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib 

In [24]:
# Load your trained models (replace with your actual model loading code)
try:
    linear_model = joblib.load('linear_regression_model.pkl')
    tree_model = joblib.load('decision_tree_model.pkl')
    forest_model = joblib.load('random_forest_model.pkl')
    scaler = joblib.load('scaler.pkl')
    label_encoders = joblib.load('label_encoders.pkl')
except:
    # If models aren't saved, create dummy models for the demo
    print("Warning: Using dummy models - for real predictions, train and save your models first")
    linear_model = LinearRegression()
    tree_model = DecisionTreeRegressor(max_depth=5, random_state=42)
    forest_model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
    scaler = StandardScaler()
    label_encoders = {}

In [25]:
# Define categorical options based on your dataset
cities = ['Mumbai', 'Delhi', 'Bangalore', 'Hyderabad', 'Chennai', 
          'Kolkata', 'Ahmedabad', 'Pune', 'Jaipur', 'Surat']
months = ['January', 'February', 'March', 'April', 'May', 'June', 
          'July', 'August', 'September', 'October', 'November', 'December']
product_types = ['Electronics', 'Clothing', 'Groceries', 'Home Appliances', 'Beauty']
marketing_channels = ['Social Media', 'TV', 'Print', 'Email', 'Influencer']
income_levels = ['Low', 'Medium', 'High']

def predict_roi(city, month, product_type, marketing_channel, income_level,
                marketing_budget, discount_rate, conversion_rate, num_sales,
                sales_revenue, customer_age):
    # Create a dataframe from the inputs
    input_data = pd.DataFrame({
        'City': [city],
        'Month': [month],
        'Product_Type': [product_type],
        'Marketing_Channel': [marketing_channel],
        'Customer_Income_Level': [income_level],
        'Marketing_Budget_INR': [marketing_budget],
        'Discount_Rate': [discount_rate],
        'Conversion_Rate': [conversion_rate],
        'Num_Sales': [num_sales],
        'Sales_Revenue_INR': [sales_revenue],
        'Customer_Age': [customer_age]
    })
    
    # Encode categorical variables
    for column in ['City', 'Month', 'Product_Type', 'Marketing_Channel', 'Customer_Income_Level']:
        if column in label_encoders:
            input_data[column] = label_encoders[column].transform(input_data[column])
        else:
            # Fallback encoding if label encoders aren't loaded
            le = LabelEncoder()
            options = eval(column.lower())  # Get the predefined options (cities, months, etc.)
            le.fit(options)
            input_data[column] = le.transform(input_data[column])
    
    # Scale numerical features
    numerical_cols = ['Marketing_Budget_INR', 'Discount_Rate', 'Conversion_Rate', 
                     'Num_Sales', 'Sales_Revenue_INR', 'Customer_Age']
    input_data[numerical_cols] = scaler.transform(input_data[numerical_cols])
    
    # Make predictions
    linear_pred = linear_model.predict(input_data)[0]
    tree_pred = tree_model.predict(input_data)[0]
    forest_pred = forest_model.predict(input_data)[0]
    
    return {
        "Linear Regression": round(linear_pred, 2),
        "Decision Tree": round(tree_pred, 2),
        "Random Forest": round(forest_pred, 2)
    }

In [26]:
# Define categorical options
cities = ['Mumbai', 'Delhi', 'Bangalore', 'Hyderabad', 'Chennai', 
          'Kolkata', 'Ahmedabad', 'Pune', 'Jaipur', 'Surat']
months = ['January', 'February', 'March', 'April', 'May', 'June', 
          'July', 'August', 'September', 'October', 'November', 'December']
product_types = ['Electronics', 'Clothing', 'Groceries', 'Home Appliances', 'Beauty']
marketing_channels = ['Social Media', 'TV', 'Print', 'Email', 'Influencer']
income_levels = ['Low', 'Medium', 'High']

def predict_roi(city, month, product_type, marketing_channel, income_level,
                marketing_budget, discount_rate, conversion_rate, num_sales,
                sales_revenue, customer_age):
    # Create a dataframe from the inputs
    input_data = pd.DataFrame({
        'City': [city],
        'Month': [month],
        'Product_Type': [product_type],
        'Marketing_Channel': [marketing_channel],
        'Customer_Income_Level': [income_level],
        'Marketing_Budget_INR': [marketing_budget],
        'Discount_Rate': [discount_rate],
        'Conversion_Rate': [conversion_rate],
        'Num_Sales': [num_sales],
        'Sales_Revenue_INR': [sales_revenue],
        'Customer_Age': [customer_age]
    })
    
    # Encode categorical variables
    for column in ['City', 'Month', 'Product_Type', 'Marketing_Channel', 'Customer_Income_Level']:
        if column in label_encoders:
            input_data[column] = label_encoders[column].transform(input_data[column])
        else:
            # Fallback encoding
            le = LabelEncoder()
            options = globals()[column.lower() + 's']  # Get the predefined options
            le.fit(options)
            input_data[column] = le.transform(input_data[column])
    
    # Scale numerical features
    numerical_cols = ['Marketing_Budget_INR', 'Discount_Rate', 'Conversion_Rate', 
                     'Num_Sales', 'Sales_Revenue_INR', 'Customer_Age']
    if hasattr(scaler, 'transform'):
        input_data[numerical_cols] = scaler.transform(input_data[numerical_cols])
    else:
        # If scaler isn't loaded, just use the raw values
        pass
    
    # Make predictions (ensure output is properly formatted)
    try:
        linear_pred = float(linear_model.predict(input_data)[0])
        tree_pred = float(tree_model.predict(input_data)[0])
        forest_pred = float(forest_model.predict(input_data)[0])
    except Exception as e:
        print(f"Prediction error: {e}")
        linear_pred = tree_pred = forest_pred = 0.0
    
    return linear_pred, tree_pred, forest_pred

In [27]:
# Create Gradio interface
with gr.Blocks(title="Marketing ROI Predictor") as demo:
    gr.Markdown("""
    # Marketing ROI Prediction Tool
    **Optimize your marketing strategy with AI-powered ROI predictions**
    """)
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Campaign Details")
            city = gr.Dropdown(cities, label="City", value="Mumbai")
            month = gr.Dropdown(months, label="Month", value="January")
            product_type = gr.Dropdown(product_types, label="Product Type", value="Electronics")
            marketing_channel = gr.Dropdown(marketing_channels, label="Marketing Channel", value="Social Media")
            income_level = gr.Dropdown(income_levels, label="Customer Income Level", value="Medium")
            customer_age = gr.Slider(22, 65, value=35, label="Customer Age")
            
        with gr.Column():
            gr.Markdown("### Marketing Metrics")
            marketing_budget = gr.Number(label="Marketing Budget (INR)", value=1500000)
            discount_rate = gr.Slider(0.05, 0.3, value=0.15, label="Discount Rate")
            conversion_rate = gr.Slider(0.01, 0.15, value=0.08, label="Conversion Rate")
            num_sales = gr.Number(label="Number of Sales", value=150000)
            sales_revenue = gr.Number(label="Sales Revenue (INR)", value=400000000)
    
    submit_btn = gr.Button("Predict ROI", variant="primary")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### ROI Predictions")
            linear_output = gr.Number(label="Linear Regression ROI", interactive=False)
            tree_output = gr.Number(label="Decision Tree ROI", interactive=False)
            forest_output = gr.Number(label="Random Forest ROI", interactive=False)
        
        with gr.Column():
            gr.Markdown("### Prediction Visualization")
            plot = gr.Plot()
    
    submit_btn.click(
        fn=predict_roi,
        inputs=[city, month, product_type, marketing_channel, income_level,
                marketing_budget, discount_rate, conversion_rate, num_sales,
                sales_revenue, customer_age],
        outputs=[linear_output, tree_output, forest_output]
    )

In [28]:
# Run the app
if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\gradio\blocks.py", line 2146, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\gradio\blocks.py", line 1664, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
 